<b>General implementation of a Quantum PCA Algorithm</b>

In [1]:
import numpy as np
import scipy
from qiskit import(
  QuantumCircuit,
  execute,
  BasicAer,
  QuantumRegister,
  ClassicalRegister)
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.synthesis import euler_angles_1q 
from math import sqrt
from scipy import random
from math import acos

Utility functions

In [3]:
def encode_eigv2(eigv):
  print(eigv)
  return (2*acos(eigv[0]), 0, 0)

def encode_covmat(covmat):
  covmat=covmat/np.trace(covmat)
  u=np.round(scipy.linalg.expm(2*np.pi*1j*np.array(covmat)),4)
  return euler_angles_1q(u)

def eigv_from(count):
  maxcount = max(count.values())
  binary_res = None
  acc=0
  for i in count.items():
    if i[1]==maxcount:
      binary_res=i[0]
  for i in range(1, len(binary_res)):
    acc+=2**(-i)*int(binary_res[i])
  return acc

Parametric implementation of the algorithm

In [4]:
def qpca(covmat, NBITS, initialeig = None, iterations=1, simulator=BasicAer.get_backend('qasm_simulator'), req_shots=8192):
  iteration=0
  last_approx=None
  while(iteration<iterations):
    circuit = QuantumCircuit(NBITS+1, NBITS+1)
    for i in range(NBITS):
      circuit.h(i)
    if iteration==0 and not initialeig:
      circuit.h(NBITS) #last bit initial guess
    elif initialeig and iteration==0:
      pars = encode_eigv2(initialeig)
      circuit.u3(pars[0],pars[1],pars[2],NBITS)
    else:
      pars = encode_eigv2(last_approx)
      circuit.u3(pars[0],pars[1],pars[2],NBITS)
    u1=encode_covmat(covmat)
    for i in range(NBITS):
      for j in range(2**i):
        circuit.cu3(u1[0],u1[1],u1[2],NBITS-1-i,NBITS)
    #inverse QFT
    for i in range(NBITS):
      circuit.h(i)
      for j in range(1,NBITS-i):
        circuit.cu1(-np.pi/(2**(j)),i,j+i)
    circuit.measure([i for i in range(NBITS+1)],[i for i in range(NBITS+1)])
    job = execute(circuit, simulator, shots=req_shots)
    result = job.result().data(circuit)
    counts = job.result().get_counts()
    keys = list(counts.keys())
    zero = sum([counts[k] for k in keys if k[0]=='0'])
    ones = sum([counts[k] for k in keys if k[0]=='1'])
    pz = sqrt(zero/(zero+ones))
    po = sqrt(ones/(zero+ones))
    last_approx = [pz, po]
    iteration+=1
  return {"results": counts, "eigv": last_approx, "lastcirc": circuit}

Generation of the covariance matrix and normalization

In [8]:
A = random.rand(2,2)
covmat = np.dot(A,A.transpose())
covmat=covmat/np.trace(covmat)
print(covmat)

[[0.673169   0.40250556]
 [0.40250556 0.326831  ]]


Target results, computed classically

In [9]:
eigval, eigvec = np.linalg.eig(covmat)
print("Eigenvalues: ")
print(eigval)
print("Eigenvectors: ")
print(eigvec)

Eigenvalues: 
[0.93817602 0.06182398]
Eigenvectors: 
[[ 0.83522577 -0.54990718]
 [ 0.54990718  0.83522577]]


Run the algorithm

In [11]:
res=qpca(covmat,7,iterations=10)
print(res["eigv"])
print(eigv_from(res["results"]))
# PLOT THE RESULTS HISTOGRAM
#plot_histogram(res["results"])
# SHOW THE CIRCUIT AT LAST ITERATION
#res["lastcirc"].draw()

[0.8286407592029854, 0.5597807536772768]
[0.8363389806920397, 0.5482126497765261]
[0.8349512289274745, 0.5503239457923851]
[0.8377244335475121, 0.5460931911656655]
[0.8351705009307979, 0.5499911220874388]
[0.8393257850426734, 0.5436287580348376]
[0.8331217179230176, 0.5530896881383706]
[0.8326820371546393, 0.5537514108334172]
[0.8313615996514393, 0.5557318513680856]
[0.8364849258952608, 0.5479899348984432]
0.9609375
